# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [3]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bilibino,68.0546,166.4372,-32.39,98,100,0.28,RU,1702947365
1,1,port elizabeth,-33.9180,25.5701,17.28,92,0,2.06,ZA,1702947365
2,2,talnakh,69.4865,88.3972,-12.66,74,100,9.87,RU,1702947365
3,3,margaret river,-33.9500,115.0667,20.27,42,0,4.89,AU,1702947366
4,4,hermanus,-34.4187,19.2345,13.78,77,0,4.39,ZA,1702947366


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [ ]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.05,
    color = "City"
)


# Display the map
map_plot_1

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [14]:
# Narrow down cities that fit criteria and drop any results with null values
filered_df = city_data_df[(city_data_df['Max Temp']>20) &
                          (city_data_df['Max Temp']<30) &
                          (city_data_df['Wind Speed']<4.5) &
                          (city_data_df['Cloudiness']<10)]

# Drop any rows with null values
goodcity_df = filered_df.dropna()

# Display sample data
goodcity_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
147,147,el porvenir,13.7617,-87.3458,21.63,60,5,3.78,HN,1702947378
158,158,kukawa,12.9234,13.5606,21.29,55,1,4.17,NG,1702947379
183,183,quelimane,-17.8786,36.8883,27.97,88,0,1.54,MZ,1702947381
203,203,ovalle,-30.5983,-71.2003,21.11,48,0,2.08,CL,1702947382
208,208,macapa,0.0389,-51.0664,28.99,74,0,3.60,BR,1702947383
280,280,monte patria,-30.6919,-70.9467,22.20,40,0,1.81,CL,1702947388
285,285,coxim,-18.5067,-54.7600,29.81,40,5,1.46,BR,1702947389
384,384,saint-philippe,-21.3585,55.7679,24.77,79,0,1.76,RE,1702947396
397,397,misliyah,17.4599,42.5572,23.16,72,5,1.57,SA,1702947397
400,400,huetamo de nunez,18.5833,-100.8833,27.31,53,6,0.61,MX,1702947397


### Step 3: Create a new DataFrame called `hotel_df`.

In [15]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = goodcity_df[['City','Country','Lat','Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
147,el porvenir,HN,13.7617,-87.3458,60,
158,kukawa,NG,12.9234,13.5606,55,
183,quelimane,MZ,-17.8786,36.8883,88,
203,ovalle,CL,-30.5983,-71.2003,48,
208,macapa,BR,0.0389,-51.0664,74,
280,monte patria,CL,-30.6919,-70.9467,40,
285,coxim,BR,-18.5067,-54.7600,40,
384,saint-philippe,RE,-21.3585,55.7679,79,
397,misliyah,SA,17.4599,42.5572,72,
400,huetamo de nunez,MX,18.5833,-100.8833,53,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [23]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel",
          "limit": 20, 
          "apiKey": geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search...")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address_json = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address_json["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search...
el porvenir - nearest hotel: No hotel found
kukawa - nearest hotel: No hotel found
quelimane - nearest hotel: Mabassa
ovalle - nearest hotel: Hotel Roxy
macapa - nearest hotel: Equatorial Hotel
monte patria - nearest hotel: Hotel Monte Cordillera
coxim - nearest hotel: Avendita
saint-philippe - nearest hotel: Le Baril
misliyah - nearest hotel: No hotel found
huetamo de nunez - nearest hotel: No hotel found
acapulco de juarez - nearest hotel: Hotel del Valle
sur - nearest hotel: Sur Hotel
juan griego - nearest hotel: Hotel Descanso Tropical
san luis de la loma - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
147,el porvenir,HN,13.7617,-87.3458,60,No hotel found
158,kukawa,NG,12.9234,13.5606,55,No hotel found
183,quelimane,MZ,-17.8786,36.8883,88,Mabassa
203,ovalle,CL,-30.5983,-71.2003,48,Hotel Roxy
208,macapa,BR,0.0389,-51.0664,74,Equatorial Hotel
280,monte patria,CL,-30.6919,-70.9467,40,Hotel Monte Cordillera
285,coxim,BR,-18.5067,-54.7600,40,Avendita
384,saint-philippe,RE,-21.3585,55.7679,79,Le Baril
397,misliyah,SA,17.4599,42.5572,72,No hotel found
400,huetamo de nunez,MX,18.5833,-100.8833,53,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.05,
    color = "City"
    hover_cols = ["City", "Hotel Name"]
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)